In [2]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm.auto import tqdm
import time
import numpy as np

data_path = '.'



In [3]:
def chrome_setting():
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver', options=chrome_options)
  return driver
driver=chrome_setting()

In [4]:
trans_list = []

In [67]:
from load_data import *
df = load_data('../dataset/train/train.csv')
df['sentence']

0        〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...
1        호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...
2        K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...
3        균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...
4        1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...
                               ...                        
32465    한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...
32466    법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...
32467    완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...
32468    중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...
32469    화순군(군수 구충곤)은 17일 동면의 이장 20여 명이 코로나 19 예방을 위해 버...
Name: sentence, Length: 32470, dtype: object

In [77]:
from urllib.request import urlopen
import json
# Crawling
def kor_to_trans(text_data, trans_lang,start_index,final_index):

  target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))

  for i in tqdm(range(start_index,final_index)): 
    
    if (i!=0)&(i%99==0):
      time.sleep(2)
      print('{}th : '.format(i), backtrans)
      np.save(data_path+'kor_to_eng_train_{}_{}.npy'.format(start_index,final_index),trans_list)
    
    try:
      query = 'https://papago.naver.com/?sk=ko&tk='+trans_lang+'&st="' + text_data[i] + '"'
      print(query)
      driver.get(query)
      time.sleep(1.5)
      element=WebDriverWait(driver, 10).until(target_present)
      time.sleep(1.5)
      backtrans = element.text 
      print(f'element text{element.text}')

      if (backtrans=='')|(backtrans==' '):
        element=WebDriverWait(driver, 10).until(target_present)
        backtrans = element.text 
        trans_list.append(backtrans)
      else:
        trans_list.append(backtrans)
    
    except BaseException as e:
      print(e)
      page = urlopen(query)
      # bytes to string
      doc = page.read().decode('utf-8')
      # string to dictionary
      dic = json.loads(doc)
      result_dict = dic['result']['trades']
      trans_list.append('')

In [78]:
stop_words = ['〈', '〉', '(', ')', '<', '>', "《", '》']
def remove_stop_words(s):
    for w in stop_words:
        s = s.replace(w, '')
        ''.join(s)
    return s

In [79]:
trans_list = []
test_sentence = [remove_stop_words(df.loc[0, 'sentence'])]
print(test_sentence)
kor_to_trans(test_sentence, 'en', 0, 1)

['Something는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 Abbey Road에 담은 노래다.']


  0%|          | 0/1 [00:00<?, ?it/s]

https://papago.naver.com/?sk=ko&tk=en&st="Something는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 Abbey Road에 담은 노래다."
HTTPConnectionPool(host='127.0.0.1', port=43698): Max retries exceeded with url: /session/826e451add93e99cc37184cf9adbb3df/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f819ac75e80>: Failed to establish a new connection: [Errno 111] Connection refused'))


In [70]:
print(trans_list)
print(' '.join(trans_list))

['']



In [ ]:
trans_list=[]
kor_to_trans(train['title'], 'en',0,10000)
np.save(data_path+'kor_to_eng_train_0_10000.npy',trans_list)

trans_list=[]
kor_to_trans(train['title'], 'en',10000,20000)
np.save(data_path+'kor_to_eng_train_10000_20000.npy',trans_list)

trans_list=[]
kor_to_trans(train['title'], 'en',20000,30000)
np.save(data_path+'kor_to_eng_train_20000_30000.npy',trans_list)

trans_list=[]
kor_to_trans(train['title'], 'en',30000,40000)
np.save(data_path+'kor_to_eng_train_30000_40000.npy',trans_list)

trans_list=[]
kor_to_trans(train['title'], 'en',40000,len(train))
np.save(data_path+'kor_to_eng_train_40000_all.npy',trans_list)

In [ ]:
eng_data0=np.load(data_path+'kor_to_eng_train_0_10000.npy')
eng_data1=np.load(data_path+'kor_to_eng_train_10000_20000.npy')
eng_data2=np.load(data_path+'kor_to_eng_train_20000_30000.npy')
eng_data3=np.load(data_path+'kor_to_eng_train_30000_40000.npy')
eng_data4=np.load(data_path+'kor_to_eng_train_40000_all.npy')

eng_data=[*eng_data0,*eng_data1,*eng_data2,*eng_data3,*eng_data4]
eng_data=pd.DataFrame(eng_data,columns=['eng_title'])

In [ ]:
back_train=pd.concat([train,eng_data],axis=1)
back_train
